In [33]:
from lenskit.algorithms import Recommender, als
from lenskit import batch, topn, util, crossfold as xf
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import pandas as pd
import joblib

from mlcomp.data.load import load_regression_train

In [34]:
df = load_regression_train()

# remove everything before 01-01-2016
df = df.loc[df['timestamp'] > 1451602800]
#df = df[:10000]


ttpair_gen = xf.partition_users(df, 2, xf.SampleFrac(0.15))
ttpair = ttpair_gen.__next__()

train = ttpair.train
test = ttpair.test

joblib.dump(train, 'train-gt16-s85.joblib')
joblib.dump(test, 'test-gt16-s15.joblib')

['test-gt16-s15.joblib']

In [35]:
bmf = als.BiasedMF(5)

bmf.fit(train)

joblib.dump(bmf, 'model-gt16-s85.joblib')

could not load LIBBLAS: Could not find module 'libblas' (or one of its dependencies). Try using the full path with constructor syntax.


['model-gt16-s85.joblib']